In [1]:
#-*-coding:utf-8-*-
import numpy as np
from sklearn.base import BaseEstimator, ClusterMixin
import networkx as nx
import pandas as pd
from PIL import  Image
import os, sys
import matplotlib.pyplot as plt
import cv2
from sklearn.preprocessing import MinMaxScaler
import xlwt  # 负责写excel
import xlrd

#爬坡
def _hill_climb(x_t, X, W=None, h=0.1, eps=1e-7):
    error = 99.
    prob = 0.
    x_l1 = np.copy(x_t)   #X(t+1)
    radius_new = 0.
    radius_old = 0.
    radius_twiceold = 0.
    iters = 0.
    while True:
        radius_thriceold = radius_twiceold
        radius_twiceold = radius_old
        radius_old = radius_new
        x_l0 = np.copy(x_l1)       #X(t)
        x_l1, density = _step(x_l0, X, W=W, h=h)
        error = density - prob
        prob = density
        radius_new = np.linalg.norm(x_l1 - x_l0)
        radius = radius_thriceold + radius_twiceold + radius_old + radius_new
        iters += 1
        if iters > 3 and error < eps:
            break
    return [x_l1, prob, radius]

#计算X(t+1)
def _step(x_l0, X, W=None, h=0.1):
    n = X.shape[0]
    d = X.shape[1]
    superweight = 0.  # superweight is the kernel X weight for each item
    x_l1 = np.zeros((1, d))
    if W is None:
        W = np.ones((n, 1))
    else:
        W = W
    for j in range(n):
        kernel = kernelize(x_l0, X[j], h, d)
        kernel = kernel * W[j] / (h ** d)
        superweight = superweight + kernel
        x_l1 = x_l1 + (kernel * X[j])
    x_l1 = x_l1 / superweight
    density = superweight / np.sum(W)
    return [x_l1, density]

#计算高斯核
def kernelize(x, y, h, degree):
    kernel = np.exp(-(np.linalg.norm(x - y) / h) ** 2. / 2.) / ((2. * np.pi) ** (degree / 2))
    return kernel
#bad code
'''def density(X,D,h,degree):
    sum1=0
    for i in range(D.shape[0]):
        k=kernelize(X,D[i],h,degree)
        sum1=sum1+k
    d=1./D.shape[0]/h**degree*sum1
    return d
def DENCLUE(D,h,xi,eps):
    rows=D.shape[0]
   A=np.zeros((rows,D.shape[1]))
    labels=-np.ones(rows)
    dens = np.zeros((rows, 1))
    degree = D.shape[1]
    for i in range(rows):
        x_mark=findattractor(D[i],D,h,eps)
        dens[i]=density(x_mark,D,h,degree)
        if dens[i]>=xi:
            A[i]=x_mark
    print A,dens
'''
class DENCLUE(BaseEstimator, ClusterMixin):
    def __init__(self, h=None, eps=1e-8, min_density=0., metric='euclidean'):
        self.h = h
        self.eps = eps
        self.min_density = min_density
        self.metric = metric

    def classify(self, X, y=None, sample_weight=None):
        if not self.eps > 0.0:
            raise ValueError("eps must be positive.")
        self.n_samples = X.shape[0]
        self.n_features =X.shape[1]
        density_attractors = np.zeros((self.n_samples, self.n_features))
        radii = np.zeros((self.n_samples, 1))
        density = np.zeros((self.n_samples, 1))

        # 构造初始值
        if self.h is None:
            self.h = np.std(X) / 5
        if sample_weight is None:
            sample_weight = np.ones((self.n_samples, 1))
        else:
            sample_weight = sample_weight
        # 初始化所有的点为noise点
        labels = -np.ones(X.shape[0])
        # 对每个样本点进行attractor和其相应密度的计算
        for i in range(self.n_samples):
            density_attractors[i], density[i], radii[i] = _hill_climb(X[i], X, W=sample_weight,
                                                                      h=self.h, eps=self.eps)
        # 构造链接图
        cluster_info = {}
        num_clusters = 0
        cluster_info[num_clusters] = {'instances': [0],
                                      'centroid': np.atleast_2d(density_attractors[0])}
        g_clusters = nx.Graph()
        for j1 in range(self.n_samples):
            g_clusters.add_node(j1, attr_dict={'attractor': density_attractors[j1], 'radius': radii[j1],
                                               'density': density[j1]})
        # 构造聚类图
        for j1 in range(self.n_samples):
            for j2 in (x for x in range(self.n_samples) if x != j1):
                if g_clusters.has_edge(j1, j2):
                    continue
                diff = np.linalg.norm(g_clusters.node[j1]['attr_dict']['attractor'] - g_clusters.node[j2]['attr_dict']['attractor'])
                if diff <= (g_clusters.node[j1]['attr_dict']['radius'] + g_clusters.node[j1]['attr_dict']['radius']):
                    g_clusters.add_edge(j1, j2)
        clusters = list(nx.connected_component_subgraphs(g_clusters))
        num_clusters = 0
        # 链接聚类
        for clust in clusters:
            # 得到attractors中的最大密度以及相应的点位信息
            max_instance = max(clust, key=lambda x: clust.node[x]['attr_dict']['density'])
            max_density = clust.node[max_instance]['attr_dict']['density']
            max_centroid = clust.node[max_instance]['attr_dict']['attractor']
            complete = False
            c_size = len(clust.nodes())
            if clust.number_of_edges() == (c_size * (c_size - 1)) / 2.:
                complete = True
            # 构造聚类字典
            cluster_info[num_clusters] = {'instances': clust.nodes(),
                                          'size': c_size,
                                          'centroid': max_centroid,
                                          'density': max_density,
                                          'complete': complete}
            # 如果类的密度小于要求，则即为noise点
            if max_density >= self.min_density:
                labels[clust.nodes()] = num_clusters
            num_clusters += 1
        self.clust_info_ = cluster_info
        self.labels_ = labels
        return self



In [11]:
r = []
g = []
b = []
brightness = []
label = []

def colorfeatures(channel):
    channel = np.array(channel)
    channel = np.mat(channel)
    channel = np.transpose(channel)
    color = []
    [h,l] = channel.shape
    for i in range(l):
        pixelVal = 0
        pixel = 0
        for j in range(h-30):
            pixelVal+=channel[j,i]*(j+1)
            pixel+=channel[j,i]
        color.append(pixelVal/pixel)  
    return color

def sizefeatures(colonySize):
    colonySize = np.array(colonySize)
    colonySize = np.mat(colonySize)
    colonySize = np.transpose(colonySize)
    [h,l] =colonySize.shape
    size=[]
    for i in range(l):
        count = 0
        for j in range(h-30):
            count+= colonySize[j,i]
        size.append(count)
    return size

def Accuracy(a,y):
    return (a==y)

imagePath1 = 'colony1/'
imagePath2 = 'colony2/'
list1 = os.listdir(imagePath1)
list2 = os.listdir(imagePath2)
for file in list1:
    if file != "":
        img=cv2.imread(imagePath1+file)
        brightness.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))  
        r.append(cv2.calcHist([img], [0], None, [256], [0.0,255.0])) 
        g.append(cv2.calcHist([img], [1], None, [256], [0.0,255.0])) 
        b.append(cv2.calcHist([img], [2], None, [256], [0.0,255.0]))
        label.append(0)
for file in list2:
    if(file !=""):
        img=cv2.imread(imagePath2+file)
        brightness.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))  
        r.append(cv2.calcHist([img], [0], None, [256], [0.0,255.0])) 
        g.append(cv2.calcHist([img], [1], None, [256], [0.0,255.0])) 
        b.append(cv2.calcHist([img], [2], None, [256], [0.0,255.0]))
        label.append(1)

colony = np.append(list1,list2)
print(colony)
print(label)
for i in range(len(brightness)):
    brightness[i]= brightness[i].mean()
scalar = MinMaxScaler(feature_range=(0, 1))
samples = np.c_[brightness,colorfeatures(r),colorfeatures(g),colorfeatures(b),sizefeatures(r)]
#samples=samples.transpose()
#print(samples)
#samples=scalar.fit_transform(samples)
#samples=samples.transpose()
picList = np.c_[list1,range(len(list1))]
d = DENCLUE(5, 0.5)
d.classify(samples[:,1:4])
a = d.clust_info_[0]['instances']

['1 (1).jpg' '1 (10).jpg' '1 (100).jpg' '1 (11).jpg' '1 (12).jpg'
 '1 (13).jpg' '1 (14).jpg' '1 (15).jpg' '1 (16).jpg' '1 (17).jpg'
 '1 (18).jpg' '1 (19).jpg' '1 (2).jpg' '1 (20).jpg' '1 (21).jpg'
 '1 (22).jpg' '1 (23).jpg' '1 (24).jpg' '1 (25).jpg' '1 (26).jpg'
 '1 (27).jpg' '1 (28).jpg' '1 (29).jpg' '1 (3).jpg' '1 (30).jpg'
 '1 (31).jpg' '1 (32).jpg' '1 (33).jpg' '1 (34).jpg' '1 (35).jpg'
 '1 (36).jpg' '1 (37).jpg' '1 (38).jpg' '1 (39).jpg' '1 (4).jpg'
 '1 (40).jpg' '1 (41).jpg' '1 (42).jpg' '1 (43).jpg' '1 (44).jpg'
 '1 (45).jpg' '1 (46).jpg' '1 (47).jpg' '1 (48).jpg' '1 (49).jpg'
 '1 (5).jpg' '1 (50).jpg' '1 (51).jpg' '1 (52).jpg' '1 (53).jpg'
 '1 (54).jpg' '1 (55).jpg' '1 (56).jpg' '1 (57).jpg' '1 (58).jpg'
 '1 (59).jpg' '1 (6).jpg' '1 (60).jpg' '1 (61).jpg' '1 (62).jpg'
 '1 (63).jpg' '1 (64).jpg' '1 (65).jpg' '1 (66).jpg' '1 (67).jpg'
 '1 (68).jpg' '1 (69).jpg' '1 (7).jpg' '1 (70).jpg' '1 (71).jpg'
 '1 (72).jpg' '1 (73).jpg' '1 (74).jpg' '1 (75).jpg' '1 (76).jpg'
 '1 (77).jpg' '1

In [9]:
out = []
count = 0
for i in range(len(colony)):
        if i in a:
            out.append("Type I colony")
            y = 0
            if(Accuracy(label[i],y)):
                count = count + 1
            else:
                print("Wrong classification！\tpicture",colony[i],":  Type I colony")
        else:
            out.append("Type II colony")
            y = 1
            if(Accuracy(label[i],y)):
                count = count + 1
            else:
                print("Wrong classification！\tpicture",colony[i],":  Type II colony")
acc = count/len(colony)
print(acc)
column_name = ['picture','Type']
out = np.c_[colony,out]
dataFrame = pd.DataFrame(out,columns = column_name)
dataFrame.to_excel("out.xls")

Wrong classification！	picture 1 (82).jpg :  Type II colony
Wrong classification！	picture 1 (83).jpg :  Type II colony
Wrong classification！	picture 1 (84).jpg :  Type II colony
Wrong classification！	picture 1 (85).jpg :  Type II colony
Wrong classification！	picture 1 (87).jpg :  Type II colony
0.9583333333333334
